In [1]:
import pyomo.environ as pyo
import pandas as pd

In [2]:
# Set time periods and products
time_periods = range(0, 12)
products = range(0, 5)

# Load data from CSV files
fixed_cost_production = pd.read_csv("fixed_cost_production.csv", index_col = [0])
variable_cost_production = pd.read_csv("variable_cost_production.csv", index_col=[0])
variable_cost_storage = pd.read_csv("variable_cost_storage.csv", index_col=[0])
demand = pd.read_csv("demand.csv", index_col=[0])
revenue = pd.read_csv("revenue.csv", index_col=[0])
storage_capacity = {0 : 580, 1: 687, 2: 599, 3: 788, 4: 194}
production_capacity = {0 : 1080, 1: 908, 2: 408, 3: 1000, 4: 403}

# Define model
model = pyo.ConcreteModel()

# Define sets
model.time_periods = pyo.Set(initialize=time_periods)
model.time_periods_with_initial = pyo.Set(initialize=[-1] + list(time_periods))
model.products = pyo.Set(initialize=products)

# Define decision variables

model.X = pyo.Var(model.products, model.time_periods, within=pyo.NonNegativeReals)
model.S = pyo.Var(model.products, model.time_periods_with_initial, within=pyo.NonNegativeReals)
model.Y = pyo.Var(model.products, model.time_periods, within=pyo.Binary)

#set domain to integer for food
for t in model.time_periods:
    model.X[0, t].domain = pyo.NonNegativeIntegers
    model.X[1, t].domain = pyo.NonNegativeIntegers
    model.S[0, t].domain = pyo.NonNegativeIntegers
    model.S[1, t].domain = pyo.NonNegativeIntegers


def init_storage_rule(model, i):
    return model.S[i, -1] == 0
model.init_storage = pyo.Constraint(model.products, rule=init_storage_rule)



# Define parameters
model.fixed_cost_production = pyo.Param(model.products, model.time_periods, initialize={(i, t): fixed_cost_production.iloc[i, t] for t in time_periods for i in products})
model.variable_cost_production = pyo.Param(model.products, model.time_periods, initialize={(i, t): variable_cost_production.iloc[i, t] for t in time_periods for i in products})
model.variable_cost_storage = pyo.Param(model.products, model.time_periods, initialize={(i, t): variable_cost_storage.iloc[i, t] for t in time_periods for i in products})
model.demand = pyo.Param(model.products, model.time_periods, initialize={(i, t): demand.iloc[i, t] for t in time_periods for i in products})
model.revenue = pyo.Param(model.products, model.time_periods, initialize={(i, t): revenue.iloc[i, t] for t in time_periods for i in products})
model.storage_capcity = pyo.Param(model.products, initialize=storage_capacity)
model.production_capcity = pyo.Param(model.products, initialize=production_capacity)
# Define objective function
model.obj = pyo.Objective(expr=sum(
    model.revenue[i, t] * (model.S[i, t-1] + model.X[i, t] - model.S[i, t])
    - model.fixed_cost_production[i, t] * model.Y[i, t] 
    - model.variable_cost_production[i, t] * model.X[i, t] 
    - model.variable_cost_storage[i, t] * model.S[i, t] 
    for t in model.time_periods for i in model.products), sense=pyo.maximize)

# Define constraints
def production_rule(model, i, t):
    return model.X[i, t] <= model.production_capcity[i] * model.Y[i, t]
model.production_constraint = pyo.Constraint(model.products, model.time_periods, rule=production_rule)

def storage_rule(model, i, t):
    return model.S[i, t] <= model.storage_capcity[i]
model.storage_constraint = pyo.Constraint(model.products, model.time_periods, rule=storage_rule)

def demand_rule(model, i, t):
    return model.S[i, t-1] + model.X[i, t] - model.S[i, t] <= model.demand[i, t]
model.demand_constraints = pyo.Constraint(model.products, model.time_periods, rule=demand_rule)


# Solve model
solver = pyo.SolverFactory("gurobi")
results = solver.solve(model)

# Print results
print("Objective function value:", model.obj())
print()
for t in model.time_periods:
    for i in model.products:
        print(f"X[{i}, {t}]: {model.X[i, t].value}, Y[{i}, {t}]: {model.Y[i, t].value}, S[{i}, {t}]: {model.S[i, t].value}")

Objective function value: 4679.031300000001

X[0, 0]: 0.0, Y[0, 0]: 0.0, S[0, 0]: -0.0
X[1, 0]: 0.0, Y[1, 0]: 0.0, S[1, 0]: -0.0
X[2, 0]: 13.879999999999999, Y[2, 0]: 1.0, S[2, 0]: 8.2
X[3, 0]: 0.0, Y[3, 0]: -0.0, S[3, 0]: 0.0
X[4, 0]: 0.0, Y[4, 0]: 0.0, S[4, 0]: 0.0
X[0, 1]: -0.0, Y[0, 1]: -0.0, S[0, 1]: -0.0
X[1, 1]: 0.0, Y[1, 1]: 0.0, S[1, 1]: -0.0
X[2, 1]: 0.0, Y[2, 1]: 0.0, S[2, 1]: 0.0
X[3, 1]: 0.0, Y[3, 1]: -0.0, S[3, 1]: 0.0
X[4, 1]: 0.0, Y[4, 1]: 0.0, S[4, 1]: 0.0
X[0, 2]: -0.0, Y[0, 2]: -0.0, S[0, 2]: -0.0
X[1, 2]: 0.0, Y[1, 2]: 0.0, S[1, 2]: -0.0
X[2, 2]: 13.680000000000009, Y[2, 2]: 1.0, S[2, 2]: 5.83
X[3, 2]: 0.0, Y[3, 2]: -0.0, S[3, 2]: 0.0
X[4, 2]: 0.0, Y[4, 2]: 0.0, S[4, 2]: 0.0
X[0, 3]: -0.0, Y[0, 3]: -0.0, S[0, 3]: -0.0
X[1, 3]: 0.0, Y[1, 3]: 0.0, S[1, 3]: -0.0
X[2, 3]: 0.0, Y[2, 3]: 0.0, S[2, 3]: 0.0
X[3, 3]: 0.0, Y[3, 3]: -0.0, S[3, 3]: 0.0
X[4, 3]: 0.0, Y[4, 3]: 0.0, S[4, 3]: 0.0
X[0, 4]: -0.0, Y[0, 4]: -0.0, S[0, 4]: -0.0
X[1, 4]: 0.0, Y[1, 4]: 0.0, S[1, 4]: -0.0
